In [1]:
import os
from kid_readout.measurement import core, legacy
from kid_readout.measurement.io import memory, readoutnc

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
#nc_filename = '/data/readout/2015-05-11_150346_compressor_noise_60_s.nc'
nc_filename = '/data/readout/2015-05-12_113832_mmw_noise_broadband.nc'
rnc = readoutnc.ReadoutNetCDF(nc_filename)

In [3]:
on_index = 0
off_index = 1
ss_channel = 0
#ss_on = legacy.sweepstream_from_rnc(rnc, sweep_group_index=on_index, timestream_group_index=on_index, channel=ss_channel)
original = legacy.sweepstream_from_rnc(rnc, sweep_group_index=off_index, timestream_group_index=off_index, channel=ss_channel)

In [4]:
name = 'original'
io = memory.IO(None)
%time core.write(original, io, name, close=False)

CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 1.74 ms


In [5]:
%time read = core.read(io, name, close=False)

CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 3.61 ms


In [6]:
def compare(a, b, verbose=False, level=0):
    keys_a = [k for k in a.__dict__ if not k.startswith('_')]
    keys_b = [k for k in b.__dict__ if not k.startswith('_')]
    for key_a in keys_a:
        assert key_a in keys_b
    for key_b in keys_b:
        assert key_b in keys_a
    keys_a.append("__class__")
    for key in keys_a:
        va = getattr(a, key)
        vb = getattr(b, key)
        if verbose:
            print("{}{}: {} {}".format("  " * level, key, type(va), repr(va)))
        if isinstance(va, core.Measurement):
            compare(va, vb, verbose=verbose, level=level+1)
        elif issubclass(va.__class__, core.MeasurementSequence):
            for n, ma in enumerate(va):
                mb = vb[n]
                compare(ma, mb, verbose=verbose, level=level+1)
        elif isinstance(va, np.ndarray):
            assert np.all(va == vb)
        else:
            assert va == vb

In [7]:
compare(original, read, verbose=True)

description: <type 'str'> 'SweepStream'
stream: <class 'kid_readout.measurement.single.Stream'> <kid_readout.measurement.single.Stream object at 0x5b93910>
  description: <type 'str'> 'ReadoutNetCDF("/data/readout/2015-05-12_113832_mmw_noise_broadband.nc").timestreams[1]'
  epoch: <type 'numpy.ndarray'> array([  1.43144558e+09,   1.43144558e+09,   1.43144558e+09, ...,
         1.43144561e+09,   1.43144561e+09,   1.43144561e+09])
  s21: <type 'numpy.ndarray'> array([-0.00401834+0.0070592j , -0.00406734+0.00702564j,
       -0.00404052+0.00684016j, ..., -0.00416106+0.00697054j,
       -0.00435822+0.00703182j, -0.00424916+0.00690136j])
  state: <class 'kid_readout.measurement.core.StateDict'> {'mmw_source': {'attenuator_turns': [9.0, 9.0]}, 'roach': {'output_attenuation': 63.04119982655925, 'hardware_state_epoch': 1431445578.672697, 'modulation_output': 0, 'hardware_state_index': 3, 'num_tones': 16, 'modulation_rate': 1, 'num_tone_samples': 2097152, 'adc_attenuation': 31.5, 'dac_attenuatio